In [1]:
import h5py
import napari
import zarr
from abbott.h5_files import *
from pathlib import Path
import imageio as io
from pathlib import Path
import pandas as pd
from skimage.filters import gaussian

In [31]:

fld = Path("Q:\carspi\ETiX-SBR-Cer1_exp1")

In [32]:
cluster_df = pd.read_csv(Path(r"Q:\carspi\ETiX_Output\240304_endpoint_clusters.csv"))  # Replace with the actual path to your CSV file


In [33]:
cluster_df = cluster_df[cluster_df['timepoint']==22]

In [23]:
cluster_df

,embryo ID,timepoint,area,major_axis_length,minor_axis_length,convex_area,centroid_local-0,centroid_local-1,eccentricity,equivalent_diameter,...,Haralick.DifferenceVariance.Mean,Haralick.DifferenceVariance.Range,Haralick.DifferenceEntropy.Mean,Haralick.DifferenceEntropy.Range,Haralick.IMC1.Mean,Haralick.IMC1.Range,Haralick.IMC2.Mean,Haralick.IMC2.Range,KMeans cluster,endpoint cluster
0,106,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.114605,0.760254,0.000000,...,0.000000,0.259397,1.000000,0.552679,1.000000,0.211959,0.000000,1.000000,0,5
1,106,1,0.031236,0.063624,0.021482,0.028534,0.053010,0.122546,0.803173,0.039182,...,0.072859,0.000000,0.914192,0.000000,0.943347,0.000000,0.107740,0.724641,0,5
2,106,2,0.053858,0.096926,0.046145,0.051619,0.035734,0.000000,0.796070,0.067064,...,0.100881,0.089989,0.877748,0.041230,0.906546,0.065787,0.174341,0.673168,0,5
3,106,3,0.106825,0.220557,0.068278,0.101035,0.216736,0.056143,0.907362,0.130825,...,0.165014,0.221619,0.804074,0.274628,0.882052,0.167436,0.205949,0.684941,0,5
4,106,4,0.134886,0.303308,0.066895,0.124229,0.312040,0.074481,1.000000,0.163794,...,0.247638,0.456935,0.703611,0.646348,0.812100,0.537011,0.307575,0.753712,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,99,18,0.806237,0.992649,0.705172,0.792647,0.734852,0.892561,0.740625,0.832137,...,0.799706,0.861139,0.167658,0.905713,0.304997,0.954173,0.801975,0.344218,5,5
2802,99,19,0.843563,0.939496,0.800791,0.835052,1.000000,0.835501,0.443354,0.865189,...,0.858777,0.895655,0.119296,0.888867,0.160838,0.893317,0.908772,0.159745,5,5
2803,99,20,0.886974,0.959869,0.847229,0.860882,0.751697,0.977619,0.365459,0.903187,...,0.875326,0.760062,0.105924,0.727674,0.155657,0.802633,0.909544,0.126657,5,5
2804,99,21,0.936324,1.000000,0.888469,0.909666,0.835003,1.000000,0.335105,0.945825,...,0.905905,1.000000,0.086592,1.000000,0.082498,1.000000,0.957212,0.125236,5,5


In [34]:
#load grid per cluster

import pandas as pd
from pathlib import Path

# Read the CSV file into a DataFrame

target_cluster = 0
embryo_ids_for_cluster = cluster_df.loc[cluster_df['endpoint clusters'] == target_cluster, 'embryo ID'].tolist()


# List to store filenames for the target cluster
files_for_cluster = []

# Iterate over the files in the folder
for fn in fld.glob("*.h5"):
    # Extract embryo ID from the filename
    embryo_id = int(fn.stem.split("_")[-1].strip('s'))

    # Check if the embryo ID belongs to the target cluster
    if embryo_id in embryo_ids_for_cluster:
        files_for_cluster.append(fn.name)

# Now files_for_cluster contains filenames for the target cluster
print(f'Files for Cluster {target_cluster}: {files_for_cluster}')



Files for Cluster 0: ['ETiX-Cer1-SBR1_s141.h5', 'ETiX-Cer1-SBR1_s143.h5', 'ETiX-Cer1-SBR1_s173.h5', 'ETiX-Cer1-SBR1_s179.h5', 'ETiX-Cer1-SBR1_s189.h5', 'ETiX-Cer1-SBR1_s218.h5', 'ETiX-Cer1-SBR1_s219.h5', 'ETiX-Cer1-SBR1_s223.h5', 'ETiX-Cer1-SBR1_s227.h5', 'ETiX-Cer1-SBR1_s231.h5', 'ETiX-Cer1-SBR1_s236.h5', 'ETiX-Cer1-SBR1_s244.h5', 'ETiX-Cer1-SBR1_s267.h5', 'ETiX-Cer1-SBR1_s273.h5', 'ETiX-Cer1-SBR1_s282.h5', 'ETiX-Cer1-SBR1_s319.h5', 'ETiX-Cer1-SBR1_s358.h5', 'ETiX-Cer1-SBR1_s381.h5', 'ETiX-Cer1-SBR1_s382.h5', 'ETiX-Cer1-SBR1_s52.h5']


In [35]:


imgs = {}
for fn in fld.glob("*.h5"):
    
    if fn.name in files_for_cluster:

        with h5py.File(fn) as f:
         
         bf=f['ch_00/1'][22]
         #mask = f['/mask_cleanup_corrected/1'][22]
         #gfp = f['ch_01/1'][...]
         #mcherry = f['ch_02/1'][...]

         site = fn.stem
         imgs[site] = bf


In [36]:
# arrange embryos on 384 well grid

import re

pattern = re.compile(r'\d{1,3}')
sorted_imgs = sorted(imgs.keys(), key=lambda x: int("".join(pattern.findall(x.split("_")[1]))))

order = pd.Series(data= np.arange(len(imgs.keys())), index= sorted_imgs)

In [37]:
#only one timepoint

from functools import partial
from typing import Union, Any
import numpy as np
from tqdm import tqdm
from numpy.typing import NDArray
from itertools import product, repeat
from typing import Callable
import pandas as pd


def rect_grid_384(sites: pd.Series, left_to_right=True, aspect_ratio=2) -> pd.DataFrame:
    n = len(sites)
    ny = 24
    nx = 16
    indices = np.array(list(product(np.arange(ny), np.arange(nx))))
    if left_to_right:
        indices = np.flip(indices, axis=1)
    return pd.DataFrame(indices[:len(sites)], columns=['iy', 'ix'], index=sites.sort_values().index)

def arrange_on_grid(raw_imgs: dict[str, NDArray],
                    order: pd.Series,
                    index_function: Callable[[pd.Series], pd.DataFrame] = rect_grid_384,
                    margin_px: int = 10) -> NDArray:
    embs = list(set(raw_imgs.keys()).intersection(set(order.index)))
    order = order.loc[embs]
    imgs = {k: v for k, v in raw_imgs.items() if k in embs}

    dtype = imgs[list(imgs.keys())[0]].dtype
    max_extent = np.array([img.shape for img in imgs.values()]).max(axis=0)

    dy_site, dx_site = max_extent
    dy_margin = dx_margin = margin_px

    site_indices = index_function(order)

    canvas_extent_yx = (site_indices.max().values + 1) * np.array(
        [dy_site + dy_margin, dx_site + dx_margin]) - np.array([dy_margin, dx_margin])

    canvas = np.zeros(canvas_extent_yx, dtype=dtype)
    for site, index in site_indices.iterrows():
        dy_current, dx_current = imgs[site].shape
        dy, dx = index.values * np.array([dy_site + dy_margin, dx_site + dx_margin])
        canvas[dy:dy + dy_current, dx:dx + dx_current] = imgs[site]

    return canvas



In [24]:
#timeline grid
from functools import partial
from typing import Union, Any
import numpy as np
from tqdm import tqdm
from numpy.typing import NDArray
from itertools import product, repeat
from typing import Callable
import pandas as pd


def rect_grid_384(sites: pd.Series, left_to_right=True, aspect_ratio=2) -> pd.DataFrame:
    n = len(sites)
    ny = 24
    nx = 16
    indices = np.array(list(product(np.arange(ny), np.arange(nx))))
    if left_to_right:
        indices = np.flip(indices, axis=1)
    return pd.DataFrame(indices[:len(sites)], columns=['iy', 'ix'], index=sites.sort_values().index)

def arrange_on_grid(raw_imgs: dict[str, NDArray],
                    order: pd.Series,
                    index_function: Callable[[pd.Series], pd.DataFrame] = rect_grid_384,
                    margin_px: int = 10) -> NDArray:
    embs = list(set(raw_imgs.keys()).intersection(set(order.index)))
    order = order.loc[embs]
    imgs = {k: v for k, v in raw_imgs.items() if k in embs}

    dtype = imgs[list(imgs.keys())[0]].dtype
    max_extent = np.array([img.shape for img in imgs.values()]).max(axis=0)

    dz_site, dy_site, dx_site = max_extent
    dy_margin = dx_margin = margin_px

    site_indices = index_function(order)

    canvas_extent_yx = (site_indices.max().values + 1) * np.array(
        [dy_site + dy_margin, dx_site + dx_margin]) - np.array([dy_margin, dx_margin])

    canvas = np.zeros((dz_site, *canvas_extent_yx), dtype=dtype)
    for site, index in site_indices.iterrows():
        _, dy_current, dx_current = imgs[site].shape
        dy, dx = index.values * np.array([dy_site + dy_margin, dx_site + dx_margin])
        canvas[:, dy:dy + dy_current, dx:dx + dx_current] = imgs[site]

    return canvas

In [38]:
canvas = arrange_on_grid(imgs, order)

viewer = napari.Viewer()
viewer.add_image(canvas)
#viewer.add_labels(canvas)

<Image layer 'canvas' at 0x1e55b2800a0>